In [1]:
import mysql.connector
import pycountry
import pandas as pd
import plotly.express as px

In [2]:
cnx = mysql.connector.connect(
  host="localhost",
  user="root",
  password="1234",
  database="cs306"
)

cursor = cnx.cursor()
cursor.execute(
    '''
    SELECT 
        p1.year AS year,
        ((p1.total_population - p2.total_population) / p2.total_population) * 100 AS population_increase_rate,
        ((c1.total_production - c2.total_production) / c2.total_production) * 100 AS production_increase_rate
    FROM (
        SELECT p.year, SUM(p.size) AS total_population
        FROM population p
        GROUP BY p.year
    ) p1
    JOIN (
        SELECT p.year, SUM(p.size) AS total_population
        FROM population p
        GROUP BY p.year
    ) p2 ON p1.year = p2.year + 1
    JOIN (
        SELECT c.year, SUM(c.production_t) AS total_production
        FROM crops c
        GROUP BY c.year
    ) c1 ON p1.year = c1.year
    JOIN (
        SELECT c.year, SUM(c.production_t) AS total_production
        FROM crops c
        GROUP BY c.year
    ) c2 ON p1.year = c2.year + 1
    ORDER BY p1.year;
    '''
)

# Fetch all the rows from the cursor as a list of tuples
result = cursor.fetchall()

# Create a DataFrame using the column names from the cursor description
df = pd.DataFrame(result, columns=["year", "population_increase_rate", "production_increase_rate"])

# Sort the DataFrame by the "year" column
df.sort_values("year", inplace=True)

# Create the line graph using Plotly Express
fig = px.line(df, x="year", y=["population_increase_rate", "production_increase_rate"], title="Average Change Rates Over the Years")
fig.update_layout(
    yaxis=dict(
        tickformat=".2f",
        title="Percentage Change"
    ),
    xaxis=dict(
        title="Year"
    ),
    legend=dict(
        title="",
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)
fig.for_each_trace(lambda t: t.update(name="Population Change Rate (%)") if t.name == "population_increase_rate" else t.update(name="Crop Production Change Rate (%)"))

fig.show()
